# Vivino project - datasets concatenation and train/test split
Author : Alphonse Doutriaux, march 2018

#### Import libs

In [1]:
import pandas as pd
import numpy as np
import unicodedata

#### Import data, concateante wines and prices, split into train and test

In [2]:
df = pd.read_csv("./data/raw_data/wines.csv", index_col='vintage_id')
prices = pd.read_csv("./data/raw_data/prices.csv", index_col='vintage_id') # a list of prices with a different order

df = df.drop_duplicates()

* For each wine, find the corresponding price

In [3]:
df['price'] = pd.Series(np.nan, index=df.index)

for index in prices.index: # for each row in df
    
    if len(prices.loc[index]) > 1:  # if two corresponding indexes in prices.index
        price_to_set = prices.loc[index].iloc[0][0] # find the corresponding price in prices
        df.loc[index, 'price'] = price_to_set # set row price with this value
        
    else: # if only one corresponding index in prices.index
        price_to_set = prices.loc[index][0] # find the corresponding price in prices
        df.loc[index, 'price'] = price_to_set # set row price with this value

* Remove outliers before data analysis (91 values)

In [4]:
indexes_to_drop = []
for index in df.index:
    prices_mean = df['price'].mean()
    prices_std = df['price'].std()
    if df.loc[index, 'price'] < prices_mean - 3* prices_std:
        indexes_to_drop.append(index)
    elif df.loc[index, 'price'] > prices_mean + 3* prices_std:
        indexes_to_drop.append(index)

* For each wine region, add the surface and production volume

In [5]:
# this function transforms "Hautes-Côtes de Nuits" into "hautes cotes de nuit"
def flatten(text):
    text = text.lower()
    text = text.replace("-", " ")
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore') 
    text = text.decode("utf-8") 
    return(text)

In [6]:
# read the table of AOC that was built using a scraping algorithm
aoc = pd.read_csv("./wikipedia_scraping/aoc_clean.csv", index_col=0)

In [7]:
# for each wine, add the corresponding regions's production volume
df['wine_region_production_vol'] = pd.Series(0, index=df.index)
for index in df.index:
    if flatten(df.loc[index, "wine_region_name"]) in aoc.name.tolist():
        corresponding_volume = aoc[aoc['name'] == flatten(df.loc[index, "wine_region_name"])].production_volume.values[0]
        df.loc[index, 'wine_region_production_vol'] = corresponding_volume
df['wine_region_production_vol'] = df['wine_region_production_vol'].replace(0, np.nan)

In [8]:
# for each wine, add the corresponding regions's surface
df['wine_region_surface'] = pd.Series(0, index=df.index)
for index in df.index:
    if flatten(df.loc[index, "wine_region_name"]) in aoc.name.tolist():
        corresponding_surface = aoc[aoc['name'] == flatten(df.loc[index, "wine_region_name"])].area.values[0]
        df.loc[index, 'wine_region_surface'] = corresponding_surface
df['wine_region_surface'] = df['wine_region_surface'].replace(0, np.nan)

* Remove the wines that do not come from France

In [9]:
df = df[df['wine_country'] == 'France']
# from 12k to 10k wines
df = df.drop(['wine_country'], axis=1)

* Split the dataset into train set and test set. As the data was imported from vivino.com with a given order, we need to shuffle the rows in order to get representative test and train sets

In [10]:
df = df.drop(indexes_to_drop, axis=0)

In [11]:
test_size = 0.2
corresponding_row_nb = round(len(df) * test_size)

In [12]:
df = df.sample(frac=1)

In [13]:
train = df[corresponding_row_nb:]
test = df[:corresponding_row_nb]

#### Export data into train.csv and test.csv

In [14]:
train.to_csv("./data/train.csv")
test.to_csv("./data/test.csv")